In [2]:
import pandas as pd
import random
import re
from tqdm.auto import tqdm

pd.options.mode.chained_assignment = None

df = pd.read_csv('NOTEEVENTS.csv')
df_dis = df[df['CATEGORY'] == 'Discharge summary']
print(len(df_dis))

/var/folders/8y/2qqnx8wn6wvdqd1p8py3xr3w0000gn/T/ipykernel_59389/2206771873.py:8: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NOTEEVENTS.csv')


59652


In [3]:
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    return y

def preprocessing(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))

    return df_less_n

df_dis = preprocessing(df_dis)
df_dis.reset_index(drop=True, inplace=True)
df_dis.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,service: addendum: radiologic s...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: ...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,service: cardiothoracic ...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,service: medicine aller...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: ...


In [5]:
# STEP 4: Notes to Sentences
from spacy.lang.en import English
tqdm.pandas()
nlp = English()  # just the language with no model
nlp.add_pipe('sentencizer')

# nlp praser may not work when there is only one token. In these cases, we just remove them as note that has length 1 usually is some random stuff

def toSentence(x):
    doc = nlp(x)
    text=[]
    try:
        for sent in doc.sents:
            st=str(sent).strip() 
            if len(st)<20:
                #a lot of abbreviation is segmented as one line. But these are all describing the previous things
                #so I attached it to the sentence before
                if len(text)!=0:
                    text[-1]=' '.join((text[-1],st))
                else:
                    text=[st]
            else:
                text.append((st))
    except:
        print(doc)
    return text

pretrain_sent=df_dis['TEXT'].progress_apply(lambda x: toSentence(x))

  0%|          | 0/59652 [00:00<?, ?it/s]

In [6]:
# get all next sentence pairs
nsp_list = []
for entry in pretrain_sent:
    for idx in range(len(entry) - 1):
        current = entry[idx] + ' ' + entry[idx + 1]
        nsp_list.append({"text_a": entry[idx], "text_b": entry[idx + 1], 'next_sentence_label': 0})

print(len(nsp_list))

# generate a random pair for each sentence
random_list = []
all_sent = [x for y in pretrain_sent for x in y]
for idx, sent in enumerate(all_sent):
    rand_int =  random.choice(range(len(all_sent)))
    while rand_int == idx:
        rand_int =  random.choice(range(len(all_sent)))
    
    random_list.append({'text_a': sent, 'text_b': all_sent[rand_int], 'next_sentence_label': 1})
    
print(len(random_list))

4862553
4922205


In [8]:
# shuffle and combine
full_list = nsp_list + random_list
random.shuffle(full_list)
final_df = pd.DataFrame(full_list)

# save to output pickle file
final_df = pd.DataFrame(full_list)
final_df.head()
final_df.to_pickle('pretrain-data.pkl')

,text_a,text_b,next_sentence_label
0,"he was treated with amiodarone, beta blocker, ...","by the time of discharge, the wound was healin...",0
1,thiamine hcl 100 mg tablet sig: one (1) tablet...,family history: noncontributory.,1
2,provisional findings impression: dxae sat 8:5...,final report indication: followup of left temp...,0
3,should you have a situation which is not life-...,followup instructions: follow-up appointment i...,0
4,*** neurology inpatient floor course *** pati...,she subsequently had two gtc which resolved wi...,0
